In [2]:
from pdf2image import convert_from_path
from transformers import pipeline
import pytesseract
import os
import json
from docx import Document
from docx2pdf import convert as docx2pdf_convert
from PIL import Image
import io

In [3]:
# Provide the full path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# READING DOCUMENT FILE
path = r"C:\Users\Himani Singla\Desktop\HIMANI\Pragra_Internship\ResumeParser_finalll\ResumeParser - Copy\data"
files = [file for file in os.listdir(path) if file.endswith((".pdf", ".docx"))]

if not files:
    print("No PDF or Word documents found in the specified directory.")
    exit()

file = files[0]
file_path = os.path.join(path, file)

if file.endswith(".pdf"):
    # CONVERTING PDF TO IMAGES
    images = convert_from_path(file_path, 500, poppler_path=r'C:\Users\Himani Singla\Desktop\HIMANI\Pragra_Internship\ResumeParser_finalll\ResumeParser - Copy\poppler-24.02.0\Library\bin')
elif file.endswith(".docx"):
    # CONVERTING DOCX TO PDF FIRST, THEN TO IMAGES
    temp_pdf_path = file_path.replace(".docx", ".pdf")
    docx2pdf_convert(file_path, temp_pdf_path)
    images = convert_from_path(temp_pdf_path, 500, poppler_path=r'C:\Users\Himani Singla\Desktop\HIMANI\Pragra_Internship\ResumeParser_finalll\ResumeParser - Copy\poppler-24.02.0\Library\bin')
    os.remove(temp_pdf_path)  # Remove temporary PDF file

# Save images
image_paths = []
iter = 1
for image in images:
    image_path = f'{path}\\output{iter}.png'
    image.save(image_path)
    image_paths.append(image_path)
    iter += 1

In [4]:
# LOADING MODEL
nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

In [11]:
# Predefined questions
questions = [
    "what is the name ",
    "What is the complete gmail id?",
    "What is the telephone number?",
    "What is the  LinkedIn ID ?",
    "Where did they work in the latest experience?",
    "what is the name of company they worked in the experience section",
    "what are the dates mention in the latest company in the experience",
    "what is work done in the latest experience?",
    "what are all the tasks done in the latest experince?",
    "what is name of the university  DURING bachelors?",
    "what is the bachelors's course?",
    "what is name of the university  DURING masters?",
    "what is the masters's course?",
    "what is name of the college or university of latest education?",
    "what is the latest course ?",

]

# Dictionary to store all answers
all_answers = []

# Loop through each question and find the answer
for question in questions:
    answer_found = False
    answer_data = {
        "question": question,
        "answer": None,
        "page": None
    }
    
    for idx, image_path in enumerate(image_paths):
        result = nlp(image_path, question)
        if result:  # Check if an answer is found
            answer_found = True
            answer_data["answer"] = result
            answer_data["page"] = idx + 1
            break
    
    if not answer_found:
        answer_data["answer"] = "No answer found in the document."
    
    all_answers.append(answer_data)

In [12]:
# Define the path to save the JSON file
output_json_path = os.path.join(path, "answers.json")

# Writing all answers to a JSON file
try:
    with open(output_json_path, "w") as json_file:
        json.dump(all_answers, json_file, indent=4)
    print(f"All answers stored in {output_json_path}")
except IOError as e:
    print(f"Failed to write to file: {e}")

All answers stored in C:\Users\Himani Singla\Desktop\HIMANI\Pragra_Internship\ResumeParser_finalll\ResumeParser - Copy\data\answers.json
